# --Pandas DataFrame--

In [15]:
import pandas as pd
df = pd.read_csv('team.csv') #Opening the team.csv for later use

In [16]:
ds =pd.read_csv('game.csv')
ds2=ds.dropna() #removing all Null variables on game dataframe

ds3= ds2[ds2['wl_home'] == 'W'].sort_values('team_id_home', ascending = False) #find all the won games per team
dsW=ds3.groupby('team_id_home').count().sort_values(['wl_home'], ascending = False) #group them and count total wins per team
ds_wins=dsW['wl_home']

dsl= ds2[ds2['wl_home'] == 'L'].sort_values('team_id_home', ascending = False) #find all loses
dsl=dsl.rename(columns={'wl_home':'loses_1981_to_2023'}) #renaming the loses column so it doesnt overwrite the wins on the merge
dsl=dsl.groupby('team_id_home').count().sort_values(['loses_1981_to_2023'], ascending = False) #group them and count total loses per team
ds_loses=dsl['loses_1981_to_2023']
ds_loses

pd.DataFrame(ds_loses)



,loses_1981_to_2023
team_id_home,
1610612764,680
1610612750,679
1610612751,662
1610612755,653
1610612746,650
1610612758,642
1610612752,620
1610612744,593
1610612749,579


In [17]:
dc = pd.read_csv('team_info_common.csv')
dc2=dc[['team_id','w','l','pts_pg','team_conference','team_division']] #getting all the columns I need for the main Dataframe to merge
dc2

,team_id,w,l,pts_pg,team_conference,team_division
0,1610612737,32,32,117.0,East,Southeast
1,1610612738,45,20,117.7,East,Atlantic
2,1610612739,40,26,112.0,East,Central
3,1610612740,31,33,114.2,West,Southwest
4,1610612741,29,36,113.0,East,Central
5,1610612742,33,32,114.0,West,Southwest
6,1610612743,45,19,117.1,West,Northwest
7,1610612744,34,31,118.0,West,Pacific
8,1610612745,15,49,110.3,West,Southwest
9,1610612746,34,33,112.7,West,Pacific


In [18]:
merged_df = pd.merge(df, ds_wins, left_on='id', right_on='team_id_home') #merging the 2 dataframes to the main one
NBA=merged_df.rename(columns={'wl_home':'wins_1981_to_2023'}) #renaming the wins column to a more fitting name
NBA =pd.merge(NBA,ds_loses,left_on='id',right_on='team_id_home') #merging the loses to the main dataframe
NBA =pd.merge(NBA,dc2,left_on='id',right_on='team_id') #merging the team info df to the main dataframe
NBA=NBA.rename(columns={'w':'current_season_wins','l':'current_season_loses','pts_pg':'avg_pts','team_conference':'region','team_division':'division'})
#renaming all the columns for the main data frame
NBA.drop('team_id',inplace=True, axis=1) #dropping any unnecessary column

# -- SCRAPPER CODE --

In [19]:
from bs4 import BeautifulSoup
import requests
import re 
import numpy as np #libralies 

url = "https://en.wikipedia.org/wiki/List_of_NBA_champions" #Url where I will Scrape my data
headers={'User-Agent': ''} 
page = requests.get(url,timeout=15, headers= headers) #getting request
print(page) #need a response of 200 for my connecting

<Response [200]>


In [20]:
soup = BeautifulSoup(page.content, "html.parser") #usinf bs4

In [21]:
soup2=soup.find_all('table') #extracting the Table i need
rows=soup2[2].find_all('tr') #finding all the rows of the table
lst=[] #list to drop all the rows and columns
for rows in rows[1:]: #for loop to run each of the table
    
    team = rows.find("td").get_text() #find the data needed for the the name of the team
        
    win = rows.find_all("td") 
    wins=win[1].get_text() #finding all the Championship Won
        
    temp = {"Team": team,"Wins": wins} 
    lst.append(temp) #appending everything to a the list

In [22]:
ls=pd.DataFrame(lst) #turning the list to a DataFrame
ls.to_csv('finals.csv') #makes a new csv(commented-out so it wont remake the CSV)

In [23]:
dfinal=pd.read_csv('finals.csv') #open the new csv
dfinal.drop('Unnamed: 0',axis=1,inplace=True) #drop a column
dfinal

,Team,Wins
0,Los Angeles Lakers,17
1,Boston Celtics,17
2,Golden State Warriors,7
3,Chicago Bulls,6
4,San Antonio Spurs,5
5,Philadelphia 76ers,3
6,Detroit Pistons,3
7,Miami Heat,3
8,New York Knicks,2
9,Houston Rockets,2


In [24]:
NBA2 =pd.merge(NBA,dfinal,left_on='full_name',right_on='Team') #merge the scraped data with the main DataFrame

In [25]:
NBA2.drop('Team',axis=1,inplace=True) #drops a duplicate column

In [26]:
NBA=NBA2.rename(columns={'Wins':'championships_won'}) #rename the new column

In [27]:
NBA.to_csv('nba.csv', index =False) #saved the final dataframe to a CSV file

In [28]:
pd.read_csv('nba.csv') #just a test of the final csv

,id,full_name,abbreviation,nickname,city,state,year_founded,wins_1981_to_2023,loses_1981_to_2023,current_season_wins,current_season_loses,avg_pts,region,division,championships_won
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949.0,880,537,32,32,117.0,East,Southeast,1
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946.0,912,510,45,20,117.7,East,Atlantic,17
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970.0,866,556,40,26,112.0,East,Central,1
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002.0,435,360,31,33,114.2,West,Southwest,0
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966.0,891,524,29,36,113.0,East,Central,6
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980.0,878,555,33,32,114.0,West,Southwest,1
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976.0,891,529,45,19,117.1,West,Northwest,1
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946.0,833,593,34,31,118.0,West,Pacific,7
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967.0,911,509,15,49,110.3,West,Southwest,2
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970.0,754,650,34,33,112.7,West,Pacific,0
